# Loading the ROS node model

First we need to load the ROS node model in Imandra. We do that using `System.mod_use` to first load the necessary messaging modules: 

In [21]:

System.mod_use ~quiet:true "src-messages/basic_types.ml";;
System.mod_use ~quiet:true "src-messages/std_msgs.ml";;
System.mod_use ~quiet:true "src-messages/rosgraph_msgs.ml";;
System.mod_use ~quiet:true "src-messages/geometry_msgs.ml";;
System.mod_use ~quiet:true "src-messages/sensor_msgs.ml";;


- : unit = ()
- : unit = ()
- : unit = ()
- : unit = ()
- : unit = ()


Then, similarly, we load the model 

In [22]:
(* Need a wrpper module stub to load the model*)
module Ros_messages = struct end;;
module Imandra_prelude = struct end;;
System.mod_use ~quiet:true "src-model/ros_model.ml";;
open Ros_model;;

module Ros_messages : sig  end
module Imandra_prelude : sig  end
- : unit = ()


The model has a `state` datatype and a `one_step : state -> state` transition function.

In [23]:
#show Ros_model.state;;

type nonrec state = {
  min_range : Basic_types.float64 option;
  incoming : Ros_model.incoming_msg option;
  outgoing : Ros_model.outgoing_msg option;
}


The `state` contains two "slots" for incoming and outgoing messages and the `min_range` value that stores the minimal range from the laser scanner received:

# Verifying error mode transition

The `min_range` state variable is an `option` and initially is `None`. Lets verify that upon receiving a `sensor_msgs/LaserScan` message, the resulting state stores some `min_range` value (not `None`). 

$$ \forall s. IsLaserScan(IncomingMessage(s)) \,\Rightarrow\, MinValue(one\_step(s)) \ne None $$

Meaning that for every state $s$, if the state contains an incoming message and this message is `LaserScan`, then the state's `min_value` is not `None` after we've called `one_step` on it.  

We can almost literally encode this formal excpresson as an Imandra `theorem`:

In [24]:
let is_laser_scan msg = 
  match msg with  Some ( Sensor _ ) -> true | _ -> false ;;
  
theorem laser_scan_fills_min_value state =
  is_laser_scan state.incoming ==> (one_step state).min_range <> None

val is_laser_scan : Ros_model.incoming_msg option -> bool = <fun>
val laser_scan_fills_min_value : Ros_model.state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.006s details Expand smt_stats (:added-eqs 73
 :conflicts 3
 :datatype-accessor-ax 24
 :datatype-constructor-ax 7
 :decisions 1
 :del-clause 2
 :max-memory 18.20
 :memory 18.20
 :mk-bool-var 76
 :mk-clause 8
 :num-allocs 616448873
 :num-checks 2
 :propagations 4
 :rlimit-count 69187
 :seq-num-reductions 1) Expand start[0.006s]
 (if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 && Is_a(Some, :var_0:.Ros_model.incoming)
 then true else false)
 ==> not
 ((if :var_0:.Ros_model.incoming = None then :var_0:
 else
 if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 then
 {Ros_model.min_range = …; Ros_model.incoming = None;
 Ros_model.outgoing = …}
 else if :var_0:.Ros_model.min_range = None then … else …).Ros_model.min_range
 = None) simplify into not
(Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 && Is_a(Some, :var_0:.Ros_model.incoming))
|| not
 ((if :var_0:.Ros_model.incoming = None then :var_0:
 else
 if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)
 then
 {Ros_model.min_range = …; Ros_model.incoming = None;
 Ros_model.outgoing = …}
 else if :var_0:.Ros_model.min_range = None then … else …).Ros_model.min_range
 = None) expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (Sensor_msgs.laserScan_range_max_3135
 (get.Ros_model.Sensor.0_3353
 … digraph "proof" {
p_211 [label="Start ((if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)\l && Is_a(Some, :var_0:.Ros_model.incoming)\l then true else false)\l ==\> not\l ((if :var_0:.Ros_model.incoming = None then :var_0:\l else\l if Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)\l then\l \{Ros_model.min_range = …; Ros_model.incoming = None;\l Ros_model.outgoing = …\}\l else if :var_0:.Ros_model.min_range = None then … else …).Ros_model.min_range\l = None)\l :time 0.006s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_211 -> p_210 [label=""];
p_210 [label="Simplify (not\l (Is_a(Ros_model.Sensor, Option.get :var_0:.Ros_model.incoming)\l && Is_a(Some, :var_0:.Ros_model.incoming))\l \|\| not\l ((if :var_0:.Ros_model.incoming = None then :var_0:\l else\l if Is_a(Ros_model.Sensor,\l Option.get :var_0:.Ros_model.incoming)\l then\l \{Ros_model.min_range = …; Ros_model.incoming = None;\l Ros_model.outgoing = …\}\l else if :var_0:.Ros_model.min_range = None then … else …).Ros_model.min_range\l = None)\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_210 -> p_209 [label=""];
p_209 -> p_z3_1239 [label="",fontname="courier",fontsize=14];
p_z3_1239 [label="(let ((a!1 (Sensor_msgs.laserScan_range_max_3135
 (get.Ros_model.Sensor.0_3353
 …",shape=box,fontname="courier",fontsize=14];p_209 [label="unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="not\l(not\l (Is_a(Ros_model.Sensor, Option.get state.Ros_model.incoming)\l && Is_a(Some, state.Ros_model.incoming))\l \|\| not\l ((if state.Ros_model.incoming = None then state\l else\l if Is_a(Ros_model.Sensor, Option.get state.Ros_model.incoming)\l then\l \{Ros_model.min_range = …; Ros_model.incoming = None;\l Ros_model.outgoing = …\}\l else if state.Ros_model.min_range = None then … else …).Ros_model.min_range\l = None))",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_1796 [label="List.fold_left anon_fun.Ros_model.get_min_range.0\l(Destruct(Ros_model.Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max\l(Destruct(Ros_model.Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_1796 [label="calls"];
}
 Load graph Loading..

One can see that the theorem is "Proven", meaning that Imandra has formally checked that this property holds for all possible input states. 

Let's also verify that if we are receiving a `rosgraph_msgs/Clock` message, and the `min_range` is less than 0.5 meters ( scaled to 50000 in our message converter ), then there is an outgoing `geometry_msgs/Twist` message with `linear.x` velocity that is less or equal to zero:  

In [25]:
let is_clock msg = 
  match msg with  Some ( Clock _ ) -> true | _ -> false ;;
  
let no_positive_outgoing_velocity state =
  let open Geometry_msgs in
  match state.outgoing with 
  | Some (Twist twist) -> twist.twist_linear.vector3_x <= 0
  | _ -> false ;;

val is_clock : Ros_model.incoming_msg option -> bool = <fun>
val no_positive_outgoing_velocity : Ros_model.state -> bool = <fun>


In [26]:
theorem clock_close_to_wall_stops state =
  (  is_clock state.incoming
  && match state.min_range with None -> false | Some m -> m < 50000 
  ) ==> no_positive_outgoing_velocity (one_step state)

val clock_close_to_wall_stops : Ros_model.state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.006s details Expand smt_stats (:added-eqs 113
 :arith-assert-lower 4
 :arith-assert-upper 2
 :arith-eq-adapter 1
 :conflicts 6
 :datatype-accessor-ax 32
 :datatype-constructor-ax 15
 :decisions 12
 :del-clause 5
 :max-memory 20.71
 :memory 20.71
 :mk-bool-var 103
 :mk-clause 6
 :num-allocs 637685704
 :num-checks 2
 :propagations 9
 :rlimit-count 70103
 :seq-num-reductions 1) Expand start[0.006s]
 (if Is_a(Ros_model.Clock, Option.get :var_0:.Ros_model.incoming)
 && Is_a(Some, :var_0:.Ros_model.incoming)
 then true else false)
 && (if :var_0:.Ros_model.min_range = None then false
 else Option.get :var_0:.Ros_model.min_range < 50000)
 ==> (if Is_a(Ros_model.Twist,
 Option.get
 (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing)
 && Is_a(Some,
 (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing)
 then
 (Destruct(Ros_model.Twist, 0,
 Option.get
 (if :var_0:.Ros_model.incoming = None then :var_0:
 else …).Ros_model.outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x
 <= 0
 else false) simplify into not
(((Is_a(Ros_model.Clock, Option.get :var_0:.Ros_model.incoming)
 && Is_a(Some, :var_0:.Ros_model.incoming))
 && not (:var_0:.Ros_model.min_range = None))
 && not (50000 <= Option.get :var_0:.Ros_model.min_range))
|| (Is_a(Ros_model.Twist,
 Option.get
 (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing)
 && Is_a(Some,
 (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing))
 && (Destruct(Ros_model.Twist, 0,
 Option.get
 (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x
 <= 0 expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (Sensor_msgs.laserScan_range_max_3135
 (get.Ros_model.Sensor.0_3353
 … digraph "proof" {
p_214 [label="Start ((if Is_a(Ros_model.Clock, Option.get :var_0:.Ros_model.incoming)\l && Is_a(Some, :var_0:.Ros_model.incoming)\l then true else false)\l && (if :var_0:.Ros_model.min_range = None then false\l else Option.get :var_0:.Ros_model.min_range \< 50000)\l ==\> (if Is_a(Ros_model.Twist,\l Option.get\l (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing)\l && Is_a(Some,\l (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing)\l then\l (Destruct(Ros_model.Twist, 0,\l Option.get\l (if :var_0:.Ros_model.incoming = None then :var_0:\l else …).Ros_model.outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \<= 0\l else false)\l :time 0.006s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_214 -> p_213 [label=""];
p_213 [label="Simplify (not\l (((Is_a(Ros_model.Clock, Option.get :var_0:.Ros_model.incoming)\l && Is_a(Some, :var_0:.Ros_model.incoming))\l && not (:var_0:.Ros_model.min_range = None))\l && not (50000 \<= Option.get :var_0:.Ros_model.min_range))\l \|\| (Is_a(Ros_model.Twist,\l Option.get\l (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing)\l && Is_a(Some,\l (if :var_0:.Ros_model.incoming = None then :var_0: else …).Ros_model.outgoing))\l && (Destruct(Ros_model.Twist, 0,\l Option.get\l (if :var_0:.Ros_model.incoming = None then :var_0:\l else …).Ros_model.outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \<= 0\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_213 -> p_212 [label=""];
p_212 -> p_z3_3056 [label="",fontname="courier",fontsize=14];
p_z3_3056 [label="(let ((a!1 (Sensor_msgs.laserScan_range_max_3135
 (get.Ros_model.Sensor.0_3353
 …",shape=box,fontname="courier",fontsize=14];p_212 [label="unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="not\l(not\l (((Is_a(Ros_model.Clock, Option.get state.Ros_model.incoming)\l && Is_a(Some, state.Ros_model.incoming))\l && not (state.Ros_